In [ ]:
import os
import zipfile
from google.colab import drive

drive.mount('/content/drive')

DRIVE_PROJECT_PATH = '/content/drive/MyDrive/skin_lession_classification'
ZIP_PATH = os.path.join(DRIVE_PROJECT_PATH, 'dataset.zip')

if not os.path.exists('/content/dataset'):
    print("Veri seti açılıyor")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')
    print("BAŞARILI! Veri seti '/content/dataset' klasörüne açıldı.")
else:
    print("Veri seti zaten hazır.")

In [ ]:
from torch import device, cuda
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import cv2

device = device("cuda:0" if cuda.is_available() else "cpu")
print(f"Eğitim şu cihazda yapılacak: {device}")

DATA_DIR = '/content/dataset'
CLASSES = sorted(os.listdir(os.path.join(DATA_DIR, 'train')))
print(f"Sınıflar: {CLASSES}")

In [ ]:
print("--- 1. ML Baseline (Random Forest) Başlıyor ---")

def load_flat_data(split_name, img_size=(64, 64)):
    # ML için resimleri küçültüp düzleştiriyoruz (vektör yapıyoruz)
    path = os.path.join(DATA_DIR, split_name)
    X, y = [], []
    for label_idx, label_name in enumerate(CLASSES):
        folder = os.path.join(path, label_name)
        for img_name in os.listdir(folder):
            img_path = os.path.join(folder, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, img_size)
                X.append(img.flatten()) # Tek satır haline getir
                y.append(label_idx)
    return np.array(X), np.array(y)

# Veriyi yükle
print("Veriler ML için hazırlanıyor...")
X_train_ml, y_train_ml = load_flat_data('train')
X_test_ml, y_test_ml = load_flat_data('test')

# Modeli Eğit
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_ml, y_train_ml)

# Test Et
y_pred_rf = rf_model.predict(X_test_ml)
print("\n>>> Random Forest Sonuçları:")
print(classification_report(y_test_ml, y_pred_rf, target_names=CLASSES))

In [ ]:
from sklearn.svm import SVC

print("\n--- 2. ML Model: SVM (Support Vector Machine) Eğitiliyor ---")

svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train_ml, y_train_ml)
# test
y_pred_svm = svm_model.predict(X_test_ml)
print(">>> SVM Sonuçları:")
print(classification_report(y_test_ml, y_pred_svm, target_names=CLASSES))

svm_model.save